In [7]:
# Tratamiento de datos
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Modelado y evaluación
# ------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

# Barra de progreso de un proceso
# ------------------------------------------------------------------------------
from tqdm import tqdm

# Configuración warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings('once')

In [ ]:
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    
    
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [8]:
df = pd.read_pickle("./datos/estandarizados.pkl")
df.head()

,instant,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,4,0,1,1,1,0,2,-0.778781,-0.632082,1.232734,-0.391485,-0.771840,654,985
1,2,4,0,1,0,2,1,2,-0.674240,-0.691755,0.469324,0.731665,-1.155857,670,801
2,3,4,0,1,0,3,1,1,-1.580704,-1.692801,-1.331685,0.729762,-1.176978,1229,1349
3,4,4,0,1,0,4,1,1,-1.560855,-1.553969,-0.266251,-0.393388,-1.200019,1454,1562
4,5,4,0,1,0,5,1,1,-1.413968,-1.450455,-1.333773,-0.054540,-1.249941,1518,1600


In [9]:
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    
    
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [10]:
# lo primero que tenemos que hacer es definir un diccionario con los hiperparámetros que queremos modificar y los valores que queremos 

param = {"max_depth": [2,4, 6], # teniendo en cuenta que teníamos overfitting tendremos que reducir la profundidad del modelo, la nuestra anterior era de 17. Bajaremos mucho este valor ya que teníamos un overfitting muy claro
        "max_features": [1,2,3,4],# calculamos en celdas anteriores, probaremos a hacer el modelo como una variable, 2, 3 y 4. Ponemos como límite el 4 ya que es el resultado de la raiz cuadrada. 
        # estos dos hiperparámetros son más difíciles de definir, pero usualmente se suelen elegir los siguientes valores
        "min_samples_split": [10, 50, 100],
        "min_samples_leaf": [10,50,100],
        "random_state" :[0]} 

In [11]:
# una vez creado el diccionario iniciaremos el modelo con GridSearch

gs = GridSearchCV(
            estimator=DecisionTreeRegressor(), # tipo de modelo que queremos hacer
            param_grid= param, # que hiperparámetros queremos que testee
            cv=10, # crossvalidation que aprendimos en la lección de regresión lineal intro. 
            verbose=-1, # para que no nos printee ningún mensaje en pantalla
            return_train_score = True, # para que nos devuelva el valor de las métricas de set de datos de entrenamiento
            scoring="neg_mean_squared_error") # la métrica que queremos que nos devuelva

In [12]:
# en df2 vamos a dejar sólo las columnas de season, workingday, cnt
df3 = df.drop(['instant',  'yr', 'mnth', 'holiday', 'weekday','temp', 'atemp', 'hum', 'windspeed', 'casual', 
       'weathersit'], axis = 1)

In [13]:
season_cambio = {2:3, 1:2, 3:1, 4:0} 

In [14]:
df3["season"]=df3["season"].map(season_cambio) 

In [15]:
# separar X e y

X = df3.drop("registered", axis = 1)
y = df3["registered"]

In [16]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [17]:
gs.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [2, 4, 6], 'max_features': [1, 2, 3, 4],
                         'min_samples_leaf': [10, 50, 100],
                         'min_samples_split': [10, 50, 100],
                         'random_state': [0]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [18]:
mejor_modelo = gs.best_estimator_
mejor_modelo

DecisionTreeRegressor(max_depth=6, max_features=3, min_samples_leaf=10,
                      min_samples_split=10, random_state=0)

In [19]:
y_pred_test_dt4 = mejor_modelo.predict(x_test)
y_pred_train_dt4 = mejor_modelo.predict(x_train)

In [21]:
dt_results4 = metricas(y_test, y_train, y_pred_test_dt4, y_pred_train_dt4, "Decision tree IV")
dt_results4

,MAE,MSE,RMSE,R2,set,modelo
0,282.751208,149622.364753,386.810502,0.948187,test,Decision tree IV
1,251.701245,113748.212839,337.265790,0.952467,train,Decision tree IV


In [23]:
rf = GridSearchCV(
            estimator=RandomForestRegressor(), # tipo de modelo que queremos hacer
            param_grid= param, # que hiperparámetros queremos que testee
            cv=10, # crossvalidation que aprendimos en la lección de regresión lineal intro. 
            verbose=-1, # para que no nos printee ningún mensaje en pantalla
            return_train_score = True, # para que nos devuelva el valor de las métricas de set de datos de entrenamiento
            scoring="neg_mean_squared_error") # la métrica que queremos que nos devuelva

In [24]:
rf.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [2, 4, 6], 'max_features': [1, 2, 3, 4],
                         'min_samples_leaf': [10, 50, 100],
                         'min_samples_split': [10, 50, 100],
                         'random_state': [0]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [25]:
mejor_modelo = rf.best_estimator_
mejor_modelo

RandomForestRegressor(max_depth=6, max_features=3, min_samples_leaf=10,
                      min_samples_split=10, random_state=0)

In [26]:
y_pred_test_dt5 = mejor_modelo.predict(x_test)
y_pred_train_dt5 = mejor_modelo.predict(x_train)

In [27]:
dt_results5 = metricas(y_test, y_train, y_pred_test_dt5, y_pred_train_dt5, "Random Forrest I")
dt_results5

,MAE,MSE,RMSE,R2,set,modelo
0,287.505296,146234.523080,382.406228,0.949360,test,Random Forrest I
1,254.152808,115736.184833,340.200213,0.951636,train,Random Forrest I


In [28]:

df_decision_results = pd.concat([dt_results4, dt_results5], axis = 0)
df_decision_results

,MAE,MSE,RMSE,R2,set,modelo
0,282.751208,149622.364753,386.810502,0.948187,test,Decision tree IV
1,251.701245,113748.212839,337.265790,0.952467,train,Decision tree IV
0,287.505296,146234.523080,382.406228,0.949360,test,Random Forrest I
1,254.152808,115736.184833,340.200213,0.951636,train,Random Forrest I


In [ ]:
### Probamos con una profundidad de 5, para ver si nos quitamos el pequeño overfiting que se aprecia. 